# Setting Limits
1) Make cuts on $E_e$
2) Make cuts on background (1 shower, 0 tracks)
3) Make cuts on $\cos(\theta_\text{NuMI})$ for both
4) Set limits
    1) For the fixed $M_\chi/M_{A'}$ case
    2) For the fixed $M_\chi$ case
    

In [4]:
import os
import uproot3 as up3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyhf
import sys
sys.path.append("../")
from src.data import load_root_file
from src.transforms import rotate_to_microboone, translate_to_microboone

dm_types = ["scalar", "fermion"]

In [3]:
# Cuts
E_e_min = 0.1 # GeV from ...
cos_theta_numi_cuts = [0.55, 0.85] # should probably study this AFTER E_e cut

### Backgrounds

In [ ]:
CUT_SHOWERS = 1
CUT_TRACKS = 0

vars = ['reco_asso_showers', 'reco_asso_tracks', 'reco_shower_dirx', 'reco_shower_diry', 'reco_shower_dirz', 'reco_shower_energy_max']
bkg_path = "../data/root/background/"

def cuts(df):
    # for now, 0 tracks, 1 shower
    df = df[df['reco_asso_showers'] == CUT_SHOWERS]
    df = df[df['reco_asso_tracks'] == CUT_TRACKS]
    return df


bkg_incryo_df = pd.concat([load_root_file("run1_NuMI_nu_overlay_with_weights.root", bkg_path, vars, cuts, 10_000, 'vertex_tree;269'), 
                          load_root_file("run1_NuMI_nu_overlay_with_weights.root", bkg_path, vars, cuts, 10_000, 'vertex_tree;268')])
bkg_dirt_df = pd.concat([load_root_file("run1_NuMI_dirt_with_weights.root", bkg_path, vars, cuts, 10_000, 'vertex_tree;127'),
                        load_root_file("run1_NuMI_dirt_with_weights.root", bkg_path, vars, cuts, 10_000, 'vertex_tree;126')])
bkg_beamoff_df = load_root_file("run1_NuMI_offbeam_full_set_sp.root", bkg_path, vars, cuts, 10_000, "singlephotonana")

bkg_total = pd.concat([bkg_incryo_df, bkg_dirt_df, bkg_beamoff_df])
display(bkg_total.head())

### Fixed $M_\chi/M_{A'}$

In [ ]:
data_dir = "../data/root/BdNMC/"

for dm_type in dm_types:
    for file in os.listdir(data_dir+dm_type+"/"):
        with up3.open(data_dir+dm_type+"/"+file) as f:
            if f.keys() == []:
                continue
            
            # Get the electron event trees

### Fixed $M_\chi$
1) Read the .out files to check for line "in Off-Shell mode" or "in On-Shell mode"
2) for now, ignore "on shell" files (we dont know that they are handelled correctly)
3) Set Limits

In [ ]:
# Get list of data files
data_dir = "../data/root/BdNMC/fixed_Mchi/"
fnames = []
for dmt in dm_types:
    for file in os.listdir(data_dir+"/"+dmt+"/"):
        if file.endswith(".root"):
            meson = file.split("_")[0]
            ma = file.split("_")[3]
            mchi = file.split("_")[5]
            fnames.append((dmt, meson, ma, mchi, file, True))

fnames = pd.DataFrame(fnames, columns=["dm_type", "meson", "ma", "mchi", "file", "Off Shell"])

# go through the .out files
# e.g nodefixed_Mchi__pi0_scalar_ma_0.09_mchi_0.001.out
out_path = "../data/out/fixed_Mchi/"
for file in fnames:
    try:
        with open(out_path+"nodefixed_Mchi__") as f:
            for line in f:
                if "Off-Shell" in line:
                    print(line)
                    # FINISH THIS!
                elif "On-Shell" in line:
                    print(file, line)
                    fnames[file, "Off Shell"] = False # idk if this works need to check later
    except:
        print("File not found", file)
